In [ ]:
import SimRunner as sr
#this class hides many details. to see everything that is happening you must study the SimRunner.py file as well as sus.#kyle_tools.multisim
bfr = sr.BitFlipRunner()

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation
%matplotlib inline

In [ ]:
#setting up parameters

p_vals={}
p_vals['lambda']=.5
p_vals['N'] = 5_000
p_vals['dt'] = 1/200
p_vals['k'] = np.pi**2
p_vals['tau'] = 1
p_vals['depth'] = 3
p_vals['location'] = 1

bfr.change_params(p_vals)


In [ ]:
%%time
bfr.run_sim(verbose=True)

In [ ]:
# if you arent sure what attribites the sim output has, this is a useful trick:
bfr.sim.output.__dict__.keys()

In [ ]:
#here is a quick trajectory trace
allstate = bfr.sim.output.all_state['states']
plt.plot(allstate[:,:,0,0].transpose());

In [ ]:
# try to create a ParamGuider class from source.kyle_tools.multisim OR make your own similar class in order to 'optimize' this bit flip. By optimize, I mean to find a protocol that has low work but also high fidelity.  Keep lambda at .5 but feel free ro change the other parameters.

# It is important also to make sure that we have the right dt, so fist come up with with a method to determine how # small of a dt is necesarry to get good convergence and how many N we need to have good statistics.